In [104]:
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import kaggle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from datetime import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer, MultiLabelBinarizer
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import NearMiss
from numpy import where
import pickle



from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, recall_score
from sklearn.model_selection import GridSearchCV, validation_curve
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
from dask.distributed import Client

My project is focused on predicitng wether an item will be purchased or removed from the cart on an ecomemrce store. This will be used to help in marketing, sales. On the supply chain side it will also give more insights into how a product is viewed by customers and how much a brand/product should be stocked or not.

### Importing and preparing the data

In [105]:
#Convert csv to DataFrame
oct19 = pd.read_csv("../data/2019-Oct.csv")
nov19 = pd.read_csv("../data/2019-Nov.csv")
dec19 = pd.read_csv("../data/2019-Dec.csv")
jan20 = pd.read_csv("../data/2020-Jan.csv")
feb20 = pd.read_csv("../data/2020-Feb.csv")

In [106]:
print(oct19.shape)
print(nov19.shape)
print(dec19.shape)
print(jan20.shape)
print(feb20.shape)
print(oct19.shape[0]+nov19.shape[0]+dec19.shape[0]+jan20.shape[0]+feb20.shape[0])

(4102283, 9)
(4635837, 9)
(3533286, 9)
(4264752, 9)
(4156682, 9)
20692840


In [107]:
ecom_df = pd.concat([oct19, nov19, dec19, jan20, feb20], axis=0)
ecom_df.head()
print(ecom_df.shape)

(20692840, 9)


In [108]:
# Dropping columns here to save memory latter on
ecom_df = ecom_df[ecom_df.event_type != 'view']
ecom_df = ecom_df[ecom_df.event_type != 'cart']

# Data Cleaning

### NAs and how to deal with them

In [116]:
#NAs in Brand
ecom_df.brand.fillna(value="unknown", axis=0, inplace=True)

In [117]:
#NAs in user_session 
ecom_df.dropna(inplace=True)

In [118]:
ecom_df.category_code.isna().sum()

0

In [119]:
ecom_df.category_code.value_counts()

appliances.environment.vacuum             16225
stationery.cartrige                       15751
apparel.glove                             15683
furniture.bathroom.bath                    4086
furniture.living_room.cabinet              2046
accessories.bag                             754
accessories.cosmetic_bag                    500
appliances.personal.hair_cutter             276
appliances.personal.massager                212
appliances.environment.air_conditioner      104
furniture.living_room.chair                  26
Name: category_code, dtype: int64

In [120]:
#Drop this column, as we will use category code instead
ecom_df.drop("category_id", axis=1, inplace = True)

KeyError: "['category_id'] not found in axis"

In [ ]:
#Get rid of negative prices
ecom_df = ecom_df[ecom_df['price']>= 0]

In [ ]:
#Set target to binary
ecom_df.event_type = ecom_df["event_type"].replace({'purchase': 1, 'remove_from_cart': 0})

# Feature Engineering

purchase per session, group session, total of price added up for each purchase in session

### Time FE

In [121]:
ecom_df['event_time'] = pd.to_datetime(ecom_df['event_time'])

In [122]:
#needs checking
dr = pd.date_range(start='2019-10-01', end='2020-02-29')
df = pd.DataFrame()
df['Date'] = dr

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())
ecom_df['holiday'] = ecom_df['event_time'].isin(holidays)

ecom_df.sample(25)

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,holiday
122182,2019-11-02 03:03:20+00:00,1,6817,apparel.glove,unknown,5.24,566576008,241b2a00-06c7-443f-b6c8-5de9d4712a46,False
3368466,2019-10-26 08:10:04+00:00,1,5855513,appliances.environment.vacuum,max,9.52,526918199,fe5f77f7-a0be-4f87-aa59-27c177cdc6db,False
3697792,2020-01-28 08:18:04+00:00,0,89343,furniture.living_room.cabinet,unknown,299.81,580840109,1923e929-599a-4f88-86ad-8cb60259b2d1,False
3962089,2019-11-27 12:24:57+00:00,0,5884577,appliances.environment.vacuum,max,47.46,564661044,bab94db2-cf7c-4e2a-8dc6-ef42f5db9736,False
463194,2019-11-04 18:24:09+00:00,0,5862021,accessories.cosmetic_bag,unknown,5.56,500179293,ee687ba8-1689-4bb8-a2d8-7511403fefda,False
3643525,2019-10-28 12:20:11+00:00,0,5877460,appliances.environment.vacuum,jessnail,21.43,259520303,def9ee98-1f78-4d02-b2dd-c643ddb62694,False
2090242,2019-10-15 23:19:30+00:00,0,5889689,apparel.glove,nitrile,9.52,479202750,53f71e28-53b9-461e-84c3-e1eb4cae7183,False
240970,2020-02-02 15:55:28+00:00,0,5889695,apparel.glove,nitrile,8.25,557560536,f909101b-ac6d-4cdb-b566-2df7e261be95,False
1247633,2020-01-11 15:38:09+00:00,0,6737,apparel.glove,unknown,5.24,574270580,480b1e02-11f1-4ddc-b572-a503decd499b,False
1166873,2019-12-09 21:57:58+00:00,0,5855508,appliances.environment.vacuum,max,95.08,580441894,215faff1-d35a-4e25-b77e-018a42ef27c8,False


In [123]:
ecom_df['month'] = pd.DatetimeIndex(ecom_df['event_time']).month
ecom_df['2020'] = pd.DatetimeIndex(ecom_df['event_time']).year
ecom_df['hour'] = pd.DatetimeIndex(ecom_df['event_time']).hour
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,holiday,month,2020,hour
264,2019-10-01 00:07:58+00:00,0,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,10,2019,0
269,2019-10-01 00:08:03+00:00,0,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,10,2019,0
274,2019-10-01 00:08:08+00:00,0,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,10,2019,0
276,2019-10-01 00:08:10+00:00,0,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,10,2019,0
278,2019-10-01 00:08:18+00:00,0,5775822,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,10,2019,0


In [124]:
#convert year to binary
ecom_df['2020'] = ecom_df['2020'].replace({2019: 0, 2020: 1})

In [125]:
ecom_df['day'] = ecom_df['event_time'].dt.dayofweek
ecom_df['day'] = ecom_df['day'].replace({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})

In [126]:
when_df = pd.DataFrame({'hour':range(1, 25)})
bins = [0,6,12,18,24]
labels = ['Night', 'Morning','Afternoon','Evening']
when_df['when'] = pd.cut(when_df['hour'], bins=bins, labels=labels, include_lowest=True)
#fix bellow
ecom_df = ecom_df.join(when_df['when'], on="hour")

In [127]:
ecom_df['month'] = ecom_df['month'].replace({1: 'January', 2: 'Febuary', 10: 'October', 11: 'November', 12: 'December'})

In [128]:
ecom_df.sample()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,holiday,month,2020,hour,day,when
4471865,2019-11-29 23:15:49+00:00,0,82957,stationery.cartrige,depilflax,2.92,564705202,c78cd822-b744-4abd-83ab-eca3b7706c70,False,November,0,23,Friday,Evening


In [129]:
#ecom_df["current_session_clicks"] = ecom_df.groupby(['user_session'])['event_type'].cumsum()

In [130]:
#need to turn the features into numeric for bellow to work
#ecom_df["current_session_event_types"] = ecom_df.groupby(['user_session'])['event_type'].cumsum()

In [131]:
#ecom_df["session_cumulative_events"] = ecom_df.groupby(ecom_df.user_session)["event_type"].cumsum()
#ecom_df.head()

# Labeling

In [132]:
encoder = LabelBinarizer()
brand_labels = pd.DataFrame(encoder.fit_transform(ecom_df['brand']), columns=encoder.classes_, index=ecom_df.index)
brand_labels.head()

,babyliss,benovy,cutrin,depilflax,dewal,domix,emil,farmavita,farmona,gehwol,...,nitrimax,oniq,polarus,profepil,runail,shifei,shik,unknown,vosev,yoko
264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
276,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
encoder = LabelBinarizer()
productid_labels = pd.DataFrame(encoder.fit_transform(ecom_df['product_id']), columns=encoder.classes_, index=ecom_df.index)
productid_labels.head()

,4923,4962,4965,4986,5395,6736,6737,6817,8098,8372,...,5926444,5926445,5926580,5926583,5926585,5926586,5928359,5929649,5929650,5929651
264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
ohe = OneHotEncoder(sparse=False, drop='first')
catco_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['category_code']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["category_code"]].columns))
catco_labels.head()

,category_code_accessories.cosmetic_bag,category_code_apparel.glove,category_code_appliances.environment.air_conditioner,category_code_appliances.environment.vacuum,category_code_appliances.personal.hair_cutter,category_code_appliances.personal.massager,category_code_furniture.bathroom.bath,category_code_furniture.living_room.cabinet,category_code_furniture.living_room.chair,category_code_stationery.cartrige
264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [135]:
ohe = OneHotEncoder(sparse=False, drop='first')
day_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['day']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["day"]].columns))
day_labels.head()

,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
264,0.0,0.0,0.0,0.0,1.0,0.0
269,0.0,0.0,0.0,0.0,1.0,0.0
274,0.0,0.0,0.0,0.0,1.0,0.0
276,0.0,0.0,0.0,0.0,1.0,0.0
278,0.0,0.0,0.0,0.0,1.0,0.0


In [136]:
ohe = OneHotEncoder(sparse=False, drop='first')
month_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['month']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["month"]].columns))
month_labels.head()

,month_Febuary,month_January,month_November,month_October
264,0.0,0.0,0.0,1.0
269,0.0,0.0,0.0,1.0
274,0.0,0.0,0.0,1.0
276,0.0,0.0,0.0,1.0
278,0.0,0.0,0.0,1.0


In [137]:
ohe = OneHotEncoder(sparse=False, drop='first')
when_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['when']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["when"]].columns))
when_labels.head()

,when_Evening,when_Morning,when_Night
264,0.0,0.0,1.0
269,0.0,0.0,1.0
274,0.0,0.0,1.0
276,0.0,0.0,1.0
278,0.0,0.0,1.0


In [138]:
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,holiday,month,2020,hour,day,when
264,2019-10-01 00:07:58+00:00,0,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,October,0,0,Tuesday,Night
269,2019-10-01 00:08:03+00:00,0,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,October,0,0,Tuesday,Night
274,2019-10-01 00:08:08+00:00,0,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,October,0,0,Tuesday,Night
276,2019-10-01 00:08:10+00:00,0,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,October,0,0,Tuesday,Night
278,2019-10-01 00:08:18+00:00,0,5775822,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,October,0,0,Tuesday,Night


In [139]:
#Join our Labeled Columns to our main DataFrame
#ecom_df = ecom_df.join(etype_labels)
ecom_df = ecom_df.join(brand_labels)
ecom_df = ecom_df.join(productid_labels)
ecom_df = ecom_df.join(catco_labels)
ecom_df = ecom_df.join(day_labels)
ecom_df = ecom_df.join(month_labels)
ecom_df = ecom_df.join(when_labels)                        
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,holiday,month,...,day_Thursday,day_Tuesday,day_Wednesday,month_Febuary,month_January,month_November,month_October,when_Evening,when_Morning,when_Night
66,2019-11-01 00:04:33+00:00,1,5766980,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,False,November,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
67,2019-11-01 00:04:33+00:00,1,5767494,stationery.cartrige,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,False,November,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
68,2019-11-01 00:04:33+00:00,1,5775813,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,False,November,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
69,2019-11-01 00:04:33+00:00,1,5775814,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,False,November,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
264,2019-10-01 00:07:58+00:00,0,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,October,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [30]:
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,holiday,month,...,day_Thursday,day_Tuesday,day_Wednesday,month_Febuary,month_January,month_November,month_October,when_Evening,when_Morning,when_Night
66,2019-11-01 00:04:33+00:00,1,5766980,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,False,November,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
67,2019-11-01 00:04:33+00:00,1,5767494,stationery.cartrige,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,False,November,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
68,2019-11-01 00:04:33+00:00,1,5775813,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,False,November,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
69,2019-11-01 00:04:33+00:00,1,5775814,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,False,November,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
264,2019-10-01 00:07:58+00:00,0,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,False,October,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


# Preprocessing Data

### Train Test Split

In [140]:
X = ecom_df.drop(["month", "hour", "day", "when", "user_id", "product_id", "category_code", "event_time", "event_type", "brand", "user_session"], axis=1)
y = ecom_df.event_type

In [32]:
X.head()

,price,holiday,2020,babyliss,benovy,cutrin,depilflax,dewal,domix,emil,...,day_Thursday,day_Tuesday,day_Wednesday,month_Febuary,month_January,month_November,month_October,when_Evening,when_Morning,when_Night
66,1.98,False,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
67,2.14,False,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
68,1.98,False,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
69,1.98,False,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
264,2.14,False,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [33]:
y.value_counts()

0    65903
1    27518
Name: event_type, dtype: int64

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Standard Scalar

In [95]:
ss = StandardScaler()
X_train_ss = pd.DataFrame(ss.fit_transform(X_train), index=X_train.index, columns=X_train.columns)  

In [96]:
ss = StandardScaler()
X_test_ss = pd.DataFrame(ss.fit_transform(X_test), index=X_test.index, columns=X_test.columns)

### SMOTE

In [141]:
sm = SMOTE(random_state=42)
X_tr_sm, y_tr_sm = sm.fit_resample(X_train, y_train)

# Models

### Logistic Regression

### LGBMClassifier

In [37]:
import lightgbm as lgb

In [38]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_train_ss,  y_train)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_train_ss)
test_preds_lgbm = lgbm.predict(X_test_ss)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

              precision    recall  f1-score   support

           0       0.74      0.96      0.83     13175
           1       0.65      0.19      0.30      5510

    accuracy                           0.73     18685
   macro avg       0.69      0.57      0.57     18685
weighted avg       0.71      0.73      0.68     18685



In [39]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_train,  y_train)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_train)
test_preds_lgbm = lgbm.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

              precision    recall  f1-score   support

           0       0.75      0.99      0.86     13175
           1       0.92      0.22      0.36      5510

    accuracy                           0.77     18685
   macro avg       0.84      0.61      0.61     18685
weighted avg       0.80      0.77      0.71     18685



In [ ]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_tr_sm,  y_tr_sm)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_tr_sm)
test_preds_lgbm = lgbm.predict(X_test_ss)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

### RandomForest

In [64]:
rf = RandomForestClassifier()

rf.fit(X_train, y_train)

# Predict on training and test sets
training_preds_rf = rf.predict(X_train)
test_preds_rf = rf.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf)
print(first_results)

              precision    recall  f1-score   support

           0       0.82      0.90      0.86     13175
           1       0.69      0.52      0.59      5510

    accuracy                           0.79     18685
   macro avg       0.75      0.71      0.73     18685
weighted avg       0.78      0.79      0.78     18685



In [41]:
rf2 = RandomForestClassifier(class_weight='balanced')

rf2.fit(X_train, y_train)

# Predict on training and test sets
training_preds_rf2 = rf2.predict(X_train)
test_preds_rf2 = rf2.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf2)
print(first_results)

              precision    recall  f1-score   support

           0       0.84      0.82      0.83     13175
           1       0.59      0.63      0.61      5510

    accuracy                           0.76     18685
   macro avg       0.71      0.72      0.72     18685
weighted avg       0.77      0.76      0.76     18685



### DecisionTreeClassifier

In [44]:
tree_clf = DecisionTreeClassifier() 

tree_clf.fit(X_train, y_train)

# Predict on training and test sets
training_preds_tclf = tree_clf.predict(X_train)
test_preds_tclf = tree_clf.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_tclf)
print(first_results)

              precision    recall  f1-score   support

           0       0.82      0.91      0.86     13175
           1       0.71      0.52      0.60      5510

    accuracy                           0.79     18685
   macro avg       0.76      0.71      0.73     18685
weighted avg       0.79      0.79      0.78     18685



In [45]:
tree_clf2 = DecisionTreeClassifier(class_weight='balanced') 

tree_clf2.fit(X_train, y_train)

# Predict on training and test sets
training_preds_tclf2 = tree_clf2.predict(X_train)
test_preds_tclf2 = tree_clf2.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_tclf2)
print(first_results)

              precision    recall  f1-score   support

           0       0.85      0.76      0.81     13175
           1       0.55      0.69      0.61      5510

    accuracy                           0.74     18685
   macro avg       0.70      0.73      0.71     18685
weighted avg       0.76      0.74      0.75     18685



In [168]:
y_test

3720468    1
3839663    0
2110095    0
1168201    0
899787     1
          ..
1052252    1
3078970    0
622452     0
2472169    0
1774235    0
Name: event_type, Length: 18685, dtype: int64

# Hyperparamter Tuning

In [65]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 95, 110, 115, 120],
    'max_features': [10, 30, 50],
    'criterion': ['gini', 'entropy'],
    'n_estimators': [100, 200, 300, 400]}

In [80]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [110, 115, 120],
    'max_features': [7],
    'criterion': ['gini'],
    'n_estimators': [400]}

In [89]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [115],
    'max_features': [30, 50],
    'criterion': ['gini'],
    'n_estimators': [400]}

In [90]:
# Instantiate the grid search model
clf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
clf.fit(X_train, y_train)
GridSearchCV(estimator=rf,
             param_grid=param_grid)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed: 14.0min remaining: 14.0min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 21.3min finished


GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True], 'criterion': ['gini'],
                         'max_depth': [115], 'max_features': [30, 50],
                         'n_estimators': [400]})

In [91]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [92]:
# Fit the grid search to the data
print(clf.best_params_)
best_grid = clf.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)
print(grid_accuracy)

{'bootstrap': True, 'criterion': 'gini', 'max_depth': 115, 'max_features': 50, 'n_estimators': 400}
Model Performance
Average Error: 0.2055 degrees.
Accuracy = -inf%.
-inf


# Best Model

In [93]:
rf3 = RandomForestClassifier(class_weight='balanced', bootstrap= True, criterion= 'gini', max_depth= 115, max_features= 50, n_estimators= 400)

rf3.fit(X_train, y_train)

# Predict on training and test sets
training_preds_rf3 = rf3.predict(X_train)
test_preds_rf3 = rf3.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf3)
print(first_results)

              precision    recall  f1-score   support

           0       0.84      0.81      0.83     13175
           1       0.59      0.63      0.61      5510

    accuracy                           0.76     18685
   macro avg       0.71      0.72      0.72     18685
weighted avg       0.77      0.76      0.76     18685



In [103]:
rf3 = RandomForestClassifier(class_weight='balanced', bootstrap= True, criterion= 'gini', max_depth= 115, max_features= 50, n_estimators= 400)

rf3.fit(X_train_ss, y_train)

# Predict on training and test sets
training_preds_rf3 = rf3.predict(X_train_ss)
test_preds_rf3 = rf3.predict(X_test_ss)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf3)
print(first_results)

              precision    recall  f1-score   support

           0       0.81      0.87      0.84     13175
           1       0.62      0.50      0.56      5510

    accuracy                           0.76     18685
   macro avg       0.72      0.69      0.70     18685
weighted avg       0.75      0.76      0.76     18685



In [142]:
rf3 = RandomForestClassifier(class_weight='balanced', bootstrap= True, criterion= 'gini', max_depth= 115, max_features= 50, n_estimators= 400)

rf3.fit(X_tr_sm, y_tr_sm)

# Predict on training and test sets
training_preds_rf3 = rf3.predict(X_tr_sm)
test_preds_rf3 = rf3.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf3)
print(first_results)

              precision    recall  f1-score   support

           0       0.84      0.84      0.84     13175
           1       0.61      0.62      0.62      5510

    accuracy                           0.77     18685
   macro avg       0.73      0.73      0.73     18685
weighted avg       0.77      0.77      0.77     18685



# test

In [144]:
rf3 = RandomForestClassifier(class_weight='balanced', bootstrap= True, criterion= 'gini', max_depth= 115, max_features= 100, n_estimators= 400)

rf3.fit(X_tr_sm, y_tr_sm)

# Predict on training and test sets
training_preds_rf3 = rf3.predict(X_tr_sm)
test_preds_rf3 = rf3.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf3)
print(first_results)

              precision    recall  f1-score   support

           0       0.84      0.84      0.84     13175
           1       0.61      0.62      0.62      5510

    accuracy                           0.77     18685
   macro avg       0.73      0.73      0.73     18685
weighted avg       0.77      0.77      0.77     18685



In [147]:
rf3 = RandomForestClassifier(bootstrap= True, criterion= 'gini', max_depth= 115, max_features= 100, n_estimators= 400)

rf3.fit(X_tr_sm, y_tr_sm)

# Predict on training and test sets
training_preds_rf3 = rf3.predict(X_tr_sm)
test_preds_rf3 = rf3.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf3)
print(first_results)

              precision    recall  f1-score   support

           0       0.84      0.84      0.84     13175
           1       0.62      0.62      0.62      5510

    accuracy                           0.77     18685
   macro avg       0.73      0.73      0.73     18685
weighted avg       0.77      0.77      0.77     18685



In [145]:

rf3 = RandomForestClassifier(class_weight='balanced', criterion= 'gini', max_depth= 115, max_features= 100, n_estimators= 400)

rf3.fit(X_train, y_train)

# Predict on training and test sets
training_preds_rf3 = rf3.predict(X_train)
test_preds_rf3 = rf3.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf3)
print(first_results)

              precision    recall  f1-score   support

           0       0.84      0.81      0.83     13175
           1       0.59      0.64      0.61      5510

    accuracy                           0.76     18685
   macro avg       0.72      0.73      0.72     18685
weighted avg       0.77      0.76      0.77     18685



In [146]:
y_tr_sm.value_counts()

1    52728
0    52728
Name: event_type, dtype: int64

# Visuals

In [176]:
#confusion matrix
plot_confusion_matrix(estimator=rf2, y_true=y_train, X=X_train)
plt.savefig("conufsion_matrix_train")

### Graph 1

In [2]:
user_df = ecom_df[['user_id','event_type','price']]

NameError: name 'ecom_df' is not defined

In [ ]:
graph_df = user_df[['user_id','price']]
graph_df = graph_df.groupby(['user_id'])['price'].sum().reset_index()
graph_df = graph_df.sort_values(by=['price'],ascending=False)
graph_df = graph_df.rename(columns={'price':'spent'})

In [ ]:
price_list = graph_df['spent'].to_list()
plt.hist(x= price_list,bins=100,log=True, color='green')
plt.xlabel('Spent amount')
plt.ylabel('Customer Count')
plt.title('Customer Spending')
plt.savefig("Customer_Spend")

### Graph 2

In [40]:
# Daily visitors number
fig, ax = plt.subplots(figsize=(15,7))
date = ecom_df.loc[:,['event_time','user_id']]
visitor_by_date = date.groupby(['event_time'])['user_id'].nunique()
visitor_by_date.plot(ax=ax)
plt.savefig("test")

### Graph 3

In [214]:
ax = sns.countplot(x=ecom_df["event_type"], data=ecom_df)
plt.savefig("Target_Value_Counts")

In [213]:
ecom_df["event_type"].value_counts()

0    65903
1    27518
Name: event_type, dtype: int64

### Graph 4

In [79]:
cart_count = ecom_df.groupby(["event_type"])["brand"].value_counts()
#({'purchase': 1, 'remove_from_cart': 0})

In [94]:
cart_p = cart_count[1]
cart_r = cart_count[0]
cart_p

brand
italwax      6845
unknown      6111
benovy       2999
max          1774
polarus      1347
depilflax    1244
nitrimax     1129
emil         1100
runail       1096
domix         928
irisk         809
nitrile       657
jessnail      441
yoko          292
gezatone      169
dewal         122
milv          119
farmona        78
profepil       62
vosev          33
gehwol         26
shik           26
nirvel         24
mavala         22
kinetics       16
naturmed       14
naomi          11
jaguar          7
cutrin          6
farmavita       4
kondor          4
kosmekka        2
masura          1
Name: brand, dtype: int64

In [128]:
cart_p = cart_count[1]
cart_r = cart_count[0]
cart_fin = pd.concat([cart_p, cart_r], axis=1)
cart_fin.columns.values[0]="Purchase"
cart_fin.columns.values[1]="Removed"
cart_fin = cart_fin.fillna(0.0)
cart_fin["Ratio"] = cart_fin.Purchase/cart_fin.Removed
cart_fin.sort_values(by="Ratio")

,Purchase,Removed,Ratio
shifei,0.0,1,0.000000
babyliss,0.0,14,0.000000
oniq,0.0,6,0.000000
kosmekka,2.0,106,0.018868
naomi,11.0,115,0.095652
vosev,33.0,280,0.117857
shik,26.0,191,0.136126
jessnail,441.0,2295,0.192157
irisk,809.0,3459,0.233883
profepil,62.0,231,0.268398


In [123]:
ax = cart_fin.plot.bar(y='Ratio', rot=0)
plt.xlabel('Brand')
plt.ylabel('Ratio P/R')
plt.xticks(rotation=90)
plt.savefig("Cart_Ratio")

### Graph ?

In [64]:
cart_count.plot.bar(figsize=(7, 5))
plt.legend(title='Value Counts', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Count')
plt.ylabel('Brand')
plt.savefig("Brand_Behaviour")

In [ ]:
sns.barplot(x='no_employees', y='Count', data=dft, hue='treatment')

In [41]:
plt.hist([bank[0], bank[1]], color=['r','b'], alpha=0.5)
plt.savefig("Brand_Behaviour")

In [ ]:
df_gb = df.groupby(['year', 'month', 'class']).size().unstack(level=2)

### Graph 5

In [66]:
grp_by_hr_event_type = ecom_df.groupby(['hour','event_type']).count()

In [68]:
layout= dict(title="Hourly Store Traffic", xaxis_title="Time of Day", yaxis_title="Number of  Users")
grp_by_hr_event_type['user_id'].unstack(1).plot(kind="bar", layout=layout)
plt.savefig("test")

In [70]:
grp_by_brand_count = ecom_df.groupby(['brand','event_type']).count()

In [72]:
layout= dict(title="Hourly Store Traffic", xaxis_title="Time of Day", yaxis_title="Number of  Users")
grp_by_brand_count['user_id'].unstack(1).plot(kind="bar", layout=layout, figsize=(16,24))
plt.savefig("test2")

# Pickle

In [42]:
# load the model from disk
loaded_model = pickle.load(open('best_model.sav', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8096333957720097
